In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import convert_all as crd

In [ ]:
# get all low_dim data
task_name = "mujoco_stack_cups"
raw_root_dir = "../demonstrations/raw"
raw_dir = f"{raw_root_dir}/{task_name}"
data = crd.raw_to_dict(
    raw_dir,
    ["obs_action.json"],
    video_file_names=None,
    flatten_mode="hdf5",
    name_converter={
        "/obs/jq": "/observations/qpos",
        "/act": "/action",
    },
    pre_process=None
)

In [ ]:
print(len(list(data.keys())))
print(data.keys())
print(data['000'].keys())
print(data['000']['/observations/qpos'][50])

In [ ]:
import os, cv2
from concurrent.futures import ThreadPoolExecutor

# merge high_dim data and save
raw_dir
video_names = ["video.mp4"]
target_root_dir = "../data/hdf5"
target_dir = f"{target_root_dir}/{task_name}"
low_dim_data = data
name_converter = {"video": "/observations/images/0"}
target_namer = lambda i: f"episode_{i}.hdf5"

compresser = crd.Compresser("jpg", [int(cv2.IMWRITE_JPEG_QUALITY), 50], True)

os.makedirs(target_dir, exist_ok=True)

# get max episode length
episode_lens = []
for low_d in low_dim_data.values():
    episode_lens.append(len(list(low_d.values())[0]))
max_pad_lenth = max(episode_lens)

# save all data
episode_names = list(low_dim_data.keys())
print(f"Max episode length: {max_pad_lenth}")
print(f"All episodes: {episode_names}")
print(f"episode number: {len(episode_names)}")

futures = []
with ThreadPoolExecutor(max_workers=25) as executor:

    def save_one(index, ep_name):
        crd.merge_video_and_save(
            low_dim_data[ep_name],
            f"{raw_dir}/{ep_name}",
            video_names,
            crd.save_dict_to_hdf5,
            name_converter,
            compresser,
            f"{target_dir}/" + target_namer(index),
            max_pad_lenth,
        )
        data.pop(ep_name)

    for index, ep_name in enumerate(episode_names):
        futures.append(executor.submit(save_one, index, ep_name))

In [ ]:
# check hdf5 data
task_name = "mujoco_stack_cups"
target_root_dir = "../data/hdf5"
target_dir = f"{target_root_dir}/{task_name}"
data = crd.hdf5_to_dict(f"{target_dir}/episode_45.hdf5")
print(data.keys())
data_flat = crd.flatten_dict(data, prefix="/")
print(data_flat.keys())

In [ ]:
# show actions
start = 120
end = start + 3

action = data_flat["/action"][start:end]
obs = data_flat["/observations/qpos"][start+1:end+1]
print(action)
print(obs)
print((obs - action))

In [ ]:
print(action[1])
print(obs[1])
print((obs[1] - action[1]) * 180 / 3.1415926)

In [ ]:
import cv2
image = data_flat["/observations/images/0"][0]
print(image.shape)
compresser = crd.Compresser("jpg", [int(cv2.IMWRITE_JPEG_QUALITY), 50], True)
image_dc = compresser.decompress(image)
print(image_dc.shape)
import matplotlib.pyplot as plt
plt.imshow(image_dc[:, :, ::-1])
plt.show()